In [32]:
import re
import os
import sys
import wget
import json
import base64
import time
import requests
import tqdm
import multiprocessing as mp
from seleniumwire import webdriver
from seleniumwire.utils import decode
from urllib.parse import unquote, quote
from pathlib import Path

In [13]:
sys.path.insert(0, str(Path('../driver').resolve()))

In [14]:
username = 'tesla'

In [20]:
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--ignore-certificate-errors-spki-list')
chrome_options.add_argument('--ignore-ssl-errors')
driver = webdriver.Chrome("../driver/chromedriver", options=chrome_options)
driver.get(f'https://twitter.com/{username}')

In [21]:
driver.requests

[Request(method='POST', url='https://accounts.google.com/ListAccounts?gpsia=1&source=ChromiumBrowser&json=standard', headers=[('content-length', '1'), ('origin', 'https://www.google.com'), ('content-type', 'application/x-www-form-urlencoded'), ('sec-fetch-site', 'none'), ('sec-fetch-mode', 'no-cors'), ('sec-fetch-dest', 'empty'), ('user-agent', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'), ('accept-encoding', 'gzip, deflate, br'), ('accept-language', 'en-US,en;q=0.9')], body=b' '),
 Request(method='GET', url='https://twitter.com/tesla', headers=[('sec-ch-ua', '" Not;A Brand";v="99", "Google Chrome";v="97", "Chromium";v="97"'), ('sec-ch-ua-mobile', '?0'), ('sec-ch-ua-platform', '"macOS"'), ('upgrade-insecure-requests', '1'), ('user-agent', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'), ('accept', 'text/html,application/xhtml+xml,app

In [22]:
userByScreenNameReq = [req for req in driver.requests if re.match('.+UserByScreenName.+', req.url)][0]
response_headers = dict(userByScreenNameReq.response.headers)
request_headers = dict(userByScreenNameReq.headers)
res_body_str = decode(userByScreenNameReq.response.body, userByScreenNameReq.response.headers.get('content-encoding', 'identity'))
res_body = json.loads(res_body_str.decode('utf-8'))
encoded_user_id = res_body['data']['user']['result']['id']
profile_image_url = res_body['data']['user']['result']['legacy']['profile_image_url_https']
profile_banner_url = res_body['data']['user']['result']['legacy']['profile_banner_url']

In [23]:
user_id = re.findall('\d+', base64.b64decode(encoded_user_id).decode('utf-8'))[0]

In [25]:
url_prefix = "https://twitter.com/i/api/graphql/a7hxKT86vkxTe8TL1qLsNw/UserMedia?variables="
variables = {
  "userId": user_id,
  "count": 10000,
  "withTweetQuoteCount": False,
  "includePromotedContent": False,
  "withSuperFollowsUserFields": True,
  "withBirdwatchPivots": False,
  "withDownvotePerspective": False,
  "withReactionsMetadata": False,
  "withReactionsPerspective": False,
  "withSuperFollowsTweetFields": True,
  "withClientEventToken": False,
  "withBirdwatchNotes": False,
  "withVoice": True,
  "withV2Timeline": False,
  "__fs_dont_mention_me_view_api_enabled": False
}
variables_str = json.dumps(variables)
url = url_prefix + quote(variables_str)

In [26]:
payload={}
response = requests.request("GET", url, headers=request_headers, data=payload)
data = response.json()

In [27]:
entries = data['data']['user']['result']['timeline']['timeline']['instructions'][0]['entries']
media = entries[0]['content']['itemContent']['tweet_results']['result']['legacy']['entities']['media'][0]
media['media_url_https']

'https://pbs.twimg.com/media/FJnYWl3WUAUCdeh.jpg'

In [28]:
def get_medias(data, entity_type: str='entities'):
    entries = data['data']['user']['result']['timeline']['timeline']['instructions'][0]['entries']
    medias = []
    for entry in entries:
        if 'itemContent' in entry['content'].keys():
            entity = entry['content']['itemContent']['tweet_results']['result']['legacy'][entity_type]
            if 'media' in entity.keys():
                medias.append(entity['media'])
    # medias = [entry['content']['itemContent']['tweet_results']['result']['legacy'][entity_type]['media'] for entry in entries if 'itemContent' in entry['content'].keys()]
    medias_flatten = [y for x in medias for y in x]
    return medias_flatten

In [29]:
medias = get_medias(data)
urls = [media['media_url_https'] for media in medias]
media_url_set = set(urls)

In [31]:
media_url_set

{'https://pbs.twimg.com/ext_tw_video_thumb/1004039791057354752/pu/img/xJOiCe6j6e1hgfdT.jpg',
 'https://pbs.twimg.com/ext_tw_video_thumb/1005485402373779457/pu/img/kaKdENBUVesWK80k.jpg',
 'https://pbs.twimg.com/ext_tw_video_thumb/1009470167833669632/pu/img/kJSizss_ap11hgdn.jpg',
 'https://pbs.twimg.com/ext_tw_video_thumb/1009559366633025536/pu/img/GlVcItcLU3oYNsYu.jpg',
 'https://pbs.twimg.com/ext_tw_video_thumb/1009889653502062593/pu/img/wUcVXvdZG-Bli32r.jpg',
 'https://pbs.twimg.com/ext_tw_video_thumb/1012497937832689665/pu/img/0W9jE-qxM30WEFzw.jpg',
 'https://pbs.twimg.com/ext_tw_video_thumb/1013893975017644035/pu/img/na5Uy3m7HkDUcgGh.jpg',
 'https://pbs.twimg.com/ext_tw_video_thumb/1013894147529310209/pu/img/cAnCAla0azg6BsNV.jpg',
 'https://pbs.twimg.com/ext_tw_video_thumb/1014221075939389440/pu/img/Ti00Dp15Qky6WDr_.jpg',
 'https://pbs.twimg.com/ext_tw_video_thumb/1015252866208952321/pu/img/3BmtML1IXt6Ci3Fu.jpg',
 'https://pbs.twimg.com/ext_tw_video_thumb/1025055932210769920/pu/img/

In [34]:
for url in media_url_set:
    wget.download(url, os.path.join('data', url.split('/')[-1]))
    time.sleep(0.3)

KeyboardInterrupt: 

In [33]:
for filepath in os.listdir('data'):
    os.remove(os.path.join('data', filepath))
def download(url):
    try:
        wget.download(url, os.path.join('data', url.split('/')[-1]))
        time.sleep(1)
    except Exception as e:
        print(e)
with mp.Pool(mp.cpu_count()) as p:
    # p.map(download, media_url_set)
    list(tqdm.tqdm(p.imap(download, list(media_url_set)[:10]), total=10))

Process SpawnPoolWorker-1:
Process SpawnPoolWorker-2:
Process SpawnPoolWorker-3:
Process SpawnPoolWorker-4:
Process SpawnPoolWorker-5:
Traceback (most recent call last):
  File "/Users/huakun/opt/anaconda3/envs/web/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/huakun/opt/anaconda3/envs/web/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/huakun/opt/anaconda3/envs/web/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/huakun/opt/anaconda3/envs/web/lib/python3.9/multiprocessing/queues.py", line 368, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'download' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "/Users/huakun/opt/anaconda3/envs/web/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/huakun/opt/anaconda3/envs/web/

KeyboardInterrupt: 